<a href="https://colab.research.google.com/github/tejusb9/Machine-Learning-in-90-days/blob/master/99Games_IAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# data analysis
import pandas as pd
import numpy as np
import random as rnd
import math
!pip install eli5
!pip install catboost
from scipy import stats
from scipy.stats import norm, skew, mode
from scipy.special import boxcox1p

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
%matplotlib inline

# machine learning
from joblib import parallel_backend
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler, LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import precision_score,f1_score

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool

import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 16.3 MB/s eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=43f8c7212a7b0b17db812c7d4c861b89155965503fcaf452ef7550397b5305fc
  Stored in directory: /root/.cache/pip/wheels/85/ac/25/ffcd87ef8f9b1eec324fdf339359be71f22612459d8c75d89c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatib

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
#games = pd.read_csv('appstore_games.csv')
games=pd.read_csv('/content/gdrive/MyDrive/colabdata/99games.csv')
games.head(5)

,install_date,player_id,country,device_brand,is_organic,var_01,var_02,var_03,var_04,var_05,...,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,revenue_d30
0,12/10/22,pl00368026c2,Indonesia,Xiaomi,1.0,12.0,75.0,45.0,1.0,5.0,...,48500.0,25.0,16.0,6.0,8174.0,19944.0,100.0,0.0,1000.0,0.00
1,07/10/22,pl00510c6d3a,Germany,samsung,0.0,10.0,65.0,45.0,NaN,1.0,...,450.0,43.0,44.0,1.0,15896.0,5810.0,38.0,0.0,7566.0,19.95
2,20/10/22,pl00bf4a1f7a,Malaysia,vivo,0.0,10.0,5.0,18.0,3.0,4.0,...,18120.0,16.0,14.0,2.0,7660.0,18020.0,112.0,0.0,939.0,0.00
3,04/10/22,pl0040d7c1bd,Thailand,NaN,0.0,10.0,11.0,17.0,NaN,3.0,...,NaN,NaN,2.0,1.0,201.0,80.0,2.0,0.0,62.0,0.00
4,11/10/22,pl00f6e4154b,Russia,HUAWEI,1.0,12.0,6.0,4.0,1.0,4.0,...,4710.0,9.0,9.0,3.0,3190.0,4710.0,6.0,0.0,252.0,0.00


In [4]:
print("Number of rows:", games.shape[0])
print("Number of columns:", games.shape[1])
print("----------------------------------------------------")
print(games.info())

Number of rows: 10580
Number of columns: 48
----------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10580 entries, 0 to 10579
Data columns (total 48 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   install_date  10578 non-null  object 
 1   player_id     10578 non-null  object 
 2   country       10578 non-null  object 
 3   device_brand  9504 non-null   object 
 4   is_organic    10579 non-null  float64
 5   var_01        10579 non-null  float64
 6   var_02        2319 non-null   float64
 7   var_03        2410 non-null   float64
 8   var_04        2085 non-null   float64
 9   var_05        1776 non-null   float64
 10  var_06        1924 non-null   float64
 11  var_07        372 non-null    float64
 12  var_08        3370 non-null   float64
 13  var_09        10579 non-null  float64
 14  var_10        7926 non-null   float64
 15  var_11        7926 non-null   float64
 16  var_12        7

In [5]:
games.describe()

,is_organic,var_01,var_02,var_03,var_04,var_05,var_06,var_07,var_08,var_09,...,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,revenue_d30
count,10579.000000,10579.000000,2319.000000,2410.000000,2085.000000,1776.000000,1924.000000,372.000000,3370.000000,10579.000000,...,6267.000000,6265.000000,7960.000000,7960.000000,7960.000000,7960.000000,7960.000000,7960.000000,10579.000000,10579.000000
mean,0.298327,10.945931,20.275981,16.915353,5.358753,7.157658,8.543139,31.362903,41.965875,353.663059,...,12821.680230,14.049162,10.411558,2.799372,3244.148744,6478.675000,41.820854,0.472920,506.956045,0.976401
std,0.457545,1.078199,174.005384,167.144070,185.993718,208.837628,197.246252,529.017026,134.307501,208.865348,...,12790.132051,55.073454,30.401240,29.384674,3071.194449,6808.070529,69.491424,29.416786,778.545295,19.436376
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,10.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,7.000000,208.035714,...,4400.000000,9.000000,5.000000,1.000000,973.750000,2080.000000,4.000000,0.000000,60.000000,0.000000
50%,0.000000,10.000000,5.000000,6.000000,1.000000,2.000000,3.000000,3.000000,19.000000,315.815789,...,9990.000000,12.000000,9.000000,2.000000,2322.000000,4410.000000,23.000000,0.000000,251.000000,0.000000
75%,1.000000,12.000000,18.000000,17.000000,1.000000,3.000000,6.000000,5.000000,49.750000,463.630682,...,17610.500000,17.000000,12.000000,3.000000,4640.750000,10090.000000,59.000000,0.000000,694.000000,0.000000
max,1.000000,12.000000,8260.000000,8169.000000,8494.000000,8803.000000,8655.000000,10207.000000,7209.000000,1191.000000,...,165652.000000,4314.000000,2619.000000,2619.000000,19801.000000,110540.000000,2619.000000,2619.000000,19801.000000,1393.800000


In [6]:
total = games.isnull().sum().sort_values(ascending=False)
percent = (total/games.isnull().count()).sort_values(ascending=False)*100
dtype = games.dtypes
na_table = pd.concat([total,percent,dtype], axis=1, keys=['Total', '%', 'Dtype'])
na_table

,Total,%,Dtype
var_07,10208,96.483932,float64
var_18,10010,94.612476,float64
var_12,9804,92.665406,float64
var_19,9698,91.663516,float64
var_22,9597,90.708885,float64
var_24,9365,88.516068,float64
var_05,8804,83.213611,float64
var_14,8800,83.175803,float64
var_06,8656,81.814745,float64
var_28,8579,81.086957,float64


In [7]:
games = games.drop(['player_id', 'var_01', 'var_02', 'var_03', 'var_04', 'var_05', 'var_06', 'var_07', 'var_08', 'var_09', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14', 'var_15', 'var_16', 'var_17', 'var_18', 'var_19', 'var_20', 'var_21', 'var_22', 'var_23', 'var_24', 'var_25', 'var_26', 'var_27', 'var_28', 'var_29', 'var_30', 'var_31', 'var_32', 'var_33', 'var_34', 'var_35', 'var_36', 'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42'], axis = 1)

In [8]:
total = games.isnull().sum().sort_values(ascending=False)
percent = (total/games.isnull().count()).sort_values(ascending=False)*100
dtype = games.dtypes
na_table = pd.concat([total,percent,dtype], axis=1, keys=['Total', '%', 'Dtype'])
na_table

,Total,%,Dtype
device_brand,1076,10.170132,object
install_date,2,0.018904,object
country,2,0.018904,object
is_organic,1,0.009452,float64
revenue_d30,1,0.009452,float64


In [10]:
games[games['revenue_d30'].isnull()]
#games.iloc[16782]

,install_date,country,device_brand,is_organic,revenue_d30
10578,NaN,NaN,NaN,NaN,NaN


In [11]:
games[games['country'].isnull()]
#games.iloc[16782]

,install_date,country,device_brand,is_organic,revenue_d30
10578,NaN,NaN,NaN,NaN,NaN
10579,NaN,NaN,1075,0.0,0.0


In [12]:
games[games['is_organic'].isnull()]
#games.iloc[16782]

,install_date,country,device_brand,is_organic,revenue_d30
10578,NaN,NaN,NaN,NaN,NaN


In [13]:
games[games['install_date'].isnull()]
#games.iloc[16782]

,install_date,country,device_brand,is_organic,revenue_d30
10578,NaN,NaN,NaN,NaN,NaN
10579,NaN,NaN,1075,0.0,0.0


In [14]:
def na_median(column):
    return column.fillna(column.median())

In [16]:
result = games.drop([10579, 10578])
print(result)

      install_date        country              device_brand  is_organic  \
0         12/10/22      Indonesia                    Xiaomi         1.0   
1         07/10/22        Germany                   samsung         0.0   
2         20/10/22       Malaysia                      vivo         0.0   
3         04/10/22       Thailand                       NaN         0.0   
4         11/10/22         Russia                    HUAWEI         1.0   
...            ...            ...                       ...         ...   
10573     14/10/22         Brazil                  motorola         0.0   
10574     14/10/22       Pakistan  INFINIX MOBILITY LIMITED         0.0   
10575     16/10/22  United States                      OPPO         0.0   
10576     15/10/22   South Africa                    HUAWEI         0.0   
10577     16/10/22    Philippines  INFINIX MOBILITY LIMITED         0.0   

       revenue_d30  
0             0.00  
1            19.95  
2             0.00  
3             0

In [17]:
games.tail(5)

,install_date,country,device_brand,is_organic,revenue_d30
10575,16/10/22,United States,OPPO,0.0,0.0
10576,15/10/22,South Africa,HUAWEI,0.0,0.0
10577,16/10/22,Philippines,INFINIX MOBILITY LIMITED,0.0,0.0
10578,NaN,NaN,NaN,NaN,NaN
10579,NaN,NaN,1075,0.0,0.0
